# Data Cleaning Notebook

## Objectives

*   Evaluate missing data
*   Clean data

## Inputs

* outputs/datasets/collection/TelcoCustomerChurn.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned

## Additional Comments | Insights | Conclusions

 

  * Drop Variables:  `['customerID', 'TotalCharges' ]`



---


# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Collected data

In [ ]:
import pandas as pd
df_raw_path = "outputs/datasets/collection/TelcoCustomerChurn.csv"
df = pd.read_csv(df_raw_path)
df.info()

# Pandas Profiling

Quick EDA

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df=df, minimal=True)
profile.to_notebook_iframe()

# Correlation and PPS Analysis

* supporting functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

def heatmap_corr(df,threshold):
  if len(df.columns) > 1:
    mask = np.zeros_like(df, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    mask[abs(df) < threshold] = True

    fig, axes = plt.subplots(figsize=(20,12))
    sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                mask=mask, cmap='viridis', annot_kws={"size": 8}, ax=axes,
                linewidth=0.5
                     )
    axes.set_yticklabels(df.columns, rotation = 0)
    plt.ylim(len(df.columns),0)
    plt.show()


def heatmap_pps(df,threshold):
    if len(df.columns) > 1:

      mask = np.zeros_like(df, dtype=np.bool)
      mask[abs(df) < threshold] = True

      fig, ax = plt.subplots(figsize=(20,12))
      ax = sns.heatmap(df, annot=True, xticklabels=True,yticklabels=True,
                        mask=mask,cmap='rocket_r', annot_kws={"size": 8},
                       linewidth=0.05,linecolor='grey')
      
      plt.ylim(len(df.columns),0)
      plt.show()



def CalculateCorrAndPPS(df):
  df_corr_spearman = df.corr(method="spearman")
  df_corr_pearson = df.corr(method="pearson")

  pps_matrix_raw = pps.matrix(df)
  pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

  pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
  print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
  print(pps_score_stats.round(3))

  return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix,CorrThreshold,PPS_Threshold):

  print("\n")
  print("* Analyze how the target variable for your ML models are correlated with other variables (features and target)")
  print("* Analyze multi colinearity, that is, how the features are correlated among themselves")

  print("\n")
  print("*** Heatmap: Spearman Correlation ***")
  print("It evaluates monotonic relationship \n")
  heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold)

  print("\n")
  print("*** Heatmap: Pearson Correlation ***")
  print("It evaluates the linear relationship between two continuous variables \n")
  heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold)

  print("\n")
  print("*** Heatmap: Power Predictive Score (PPS) ***")
  print(f"PPS detects linear or non-linear relationships between two columns.\n"
        f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
  heatmap_pps(df=pps_matrix,threshold=PPS_Threshold)

* Calculate Correlations and Power Predictive Score

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

* Display at Heatmaps

In [ ]:
DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix,
                  CorrThreshold=0.2, PPS_Threshold=0.2)

# Data Cleaning

## Assessing Missing Data Levels

* Custom function to display missing data levels in a dataframe, it shows the aboslute levels, relative levels and data type

In [ ]:
def EvaluateMissingData(df):
  missing_data_absolute = df.isnull().sum()
  missing_data_percentage = round(missing_data_absolute/len(df)*100 , 2)
  df_missing_data = (pd.DataFrame(
                          data= {"RowsWithMissingData": missing_data_absolute,
                                 "PercentageOfDataset": missing_data_percentage,
                                 "DataType":df.dtypes}
                                  )
                    .sort_values(by=['PercentageOfDataset'],ascending=False)
                    .query("PercentageOfDataset > 0")
                    )

  return df_missing_data

Check missing data levels for collected dataset

In [ ]:
EvaluateMissingData(df)

## Data Cleaning Spreadsheet Summary

* Go to your spreadsheet at your Google Drive to list **potential approaches to handle missing data**!

## Dealing with Missing Data


* It is assumed that you  already:
  * assessed the missing data levels, 
  * did a quick EDA, 
  * checked correlation (pearson, spearman),
  * checked power predictive score.
* So you are aware of the variables to work on

---

* **Strategy**
* First, for all variables you need to imput missing data, write potential imputation approach for data cleaning.
  * Over the course, you saw multiple approaches for dealing with missing data, like DropVariables, DropNA, Imput with mean/median/mode, Imput the most frequent item etc

* Then, you will **iterate the steps below across different imputation approaches**, so at the end you will have dealt with all variables with missing data

  * 1 -  Select a **imputation approach**
  * 2 - Select **variables** to apply the approach
  * 3 - Create a **separate dataframe** applying this imputation approach to the selected variables
  * 4 - **Compare** this new dataset with initial dataset to validate/assess the effect on distribution on variables
  * 5 - **If** you are satisfied, **apply** the selected imputation approach to the initial dataframe
  * 6 - **Evaluate** if you have more variables to deal. If yes, iterate. If not, you are done.

---

### Custom function

Custom function to evaluate variables distribution before and after applying a given method. 

In [ ]:
import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt

def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # select variables in which the given data cleaning method was not applied  
  variables_not_applied_with_method = [x for x in df_cleaned.columns if x not in variables_applied_with_method]

  # scan over variables, 
    # first on variables that you applied the method
    # if variable is numerical, plots histogram, if categorical, plots barplot
  for set_of_variables in [variables_applied_with_method,variables_not_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:   
        # it is categorical variable: barplot

        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        # use a statistical test to inform if there is significant change
        plt.figure(figsize=(20, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"]).set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: 
        # it is numerical variable: histogram

        # use a statistical test to inform if there is significant change
        plt.figure(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step")
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step").set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1


### Data Cleaning Summary

List here the imputation approaches you want initially to try.
* Drop


**The list above is your guide, your map to know in which stage you are in the data cleaning process**

### Split Train and Test Set

* You have to split train and test set for cleaning the data
  * Unless you consider only Drop Variables and Drop Rows, which is not the case.
  * Hint: in the majority of the time in the workplace, you will need to split into train and test set

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['Churn'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

### Drop Variables


* Hint: you may drop Variables with more than 80% of missing data, since these variables will likely not add much value. However, this is not the case in this dataset
* Step 1: imputation approach: **Drop Variables**
* Step 2: Select variables to apply the imputation approach


In [ ]:
variables_method = ['customerID', 'TotalCharges' ]

print(f"* {len(variables_method)} variables to drop \n\n"
    f"{variables_method}")


* We are dropping `CustomerID`, since it is a unique identifier to each customer, it doesn't add information to the dataset as it is
* We are dropping `TotalCharges` since for the context of the ML projects, a prospect doesn't have `TotalCharges`

* Step 3: Create a separate dataframe applying this imputation approach to the selected variables

In [ ]:
from feature_engine.selection import DropFeatures
imputer = DropFeatures(features_to_drop=variables_method)
imputer.fit(TrainSet)
df_method = imputer.transform(TrainSet)

* Step 4: Assess the effect on variable's distribution

* In this case, no effect on variables distribution, since you are not removing rows, but columns
* The effect might be losing features that might have a relevant impact in your machine learning model.

* Step 5: If you are statisfied, apply the imputation approach in your dataframe

In [ ]:
from feature_engine.selection import DropFeatures
imputer = DropFeatures(features_to_drop=variables_method)
imputer.fit(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet) , imputer.transform(TestSet)

* Step 6: Evaluate if you have more variables to deal. If yes, iterate. If not, you are done.

In [ ]:
EvaluateMissingData(TrainSet)

# Push cleaned data to Repo

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)


## Train Set

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv",index=False)

## Test Set

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv",index=False)

## **Push** generated/new files from this Session to GitHub repo

* You may now go to "Push generated/new files from this session to GitHub Repo" section and push these files to the repo

* Git status

In [ ]:
! git status

* Git commit

In [ ]:
CommitMsg = "add-files-data-cleaning"
!git add .
!git commit -m {CommitMsg}

* Git Push

In [ ]:
!git push origin main

Well done!